# Deploy model

In [1]:
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
import pandas as pd
import numpy as np
import imp
import pickle
import os

In [2]:
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

Load the test dataset and retain just one row. This record will be used to create and input schema for the web service. It will also allow us to simulate invoking the web service with features for one hour period and generating a demand forecast for this hour.

In [6]:
test_df = pd.read_csv(aml_dir + 'nyc_demand_test.csv', parse_dates=['timeStamp'])
test_df = test_df.drop(['demand', 'timeStamp'], axis=1).copy().iloc[[0]]
test_df

,precip,temp,hour,month,dayofweek,demand_lag1,demand_lag2,demand_lag3,demand_lag4,demand_lag5,demand_lag6,demand_lag12,demand_lag24,demand_rmean_day,demand_rmean_wk,temp_rmean_day,temp_rmean_wk
0,0.0,74.63,0,6,4,6912.7,7332.625,7576.558,7603.008,7788.292,8102.142,8161.383,6247.433,7205.5525,6927.0806,76.0283,74.159


Load model from disk. Enter the name of the model to deploy.

In [39]:
model_name = "linear_regression"
with open(aml_dir + model_name + '.pkl', 'rb') as f:
    mod = pickle.load(f)
    
if not os.path.exists('models'):
    os.makedirs('models')
with open('./models/model.pkl', 'wb') as f:
    mod = pickle.dump(mod, f)

In [10]:
mod

Pipeline(memory=None,
     steps=[('onehot', OneHotEncoder(categorical_features=[2, 3, 4], dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)), ('rfecv', RFECV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
   estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
   n_jobs=-1, scoring='neg_mean_squared_error', step=1, verbose=2))])

Apply model to predict test record

In [11]:
mod.predict(test_df)

array([ 6472.48906406])

### Author a realtime web service

Create a score.py script which implements the scoring function to run inside the web service. Change model_name variable as required.

In [42]:
%%writefile score.py
# The init and run functions will load and score the input using the saved model.
# The score.py file will be included in the web service deployment package.
def init():
    import pickle
    import os
    global model
    
    with open('./models/model.pkl', 'rb') as f:
        model = pickle.load(f)
    
def run(input_df):
    try:
        if (input_df.shape != (1,17)):
            return 'Bad imput: Expecting dataframe of shape (1,17)'
        else:
            pred = model.predict(input_df)
            return int(pred)
    except Exception as e:
        return(str(e))

Overwriting score.py


This script will be written to your current working directory:

In [23]:
os.getcwd()

'C:\\Users\\anta\\AppData\\Local\\Temp\\azureml_runs\\TimeSeriesForecastingTutorial_1505320068078'

### Test the *init* and *run* functions

In [18]:
import score

Use this command to reimport the score module if you subsequently make changes to it in this session.

In [43]:
imp.reload(score)

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))


<module 'score' from 'C:\\Users\\anta\\AppData\\Local\\Temp\\azureml_runs\\TimeSeriesForecastingTutorial_1505320068078\\score.py'>

In [44]:
score.init()
score.run(test_df)

6472

### Create Web Service Schema
The web service schema provides details on the required structure of the input data as well as the data types of each column.

In [34]:
inputs = {"input_df": SampleDefinition(DataTypes.PANDAS, test_df)}
generate_schema(run_func=score.run, inputs=inputs, filepath="service_schema.json")

{'input': {'input_df': {'internal': 'gANjYXp1cmVtbC5hcGkuc2NoZW1hLnBhbmRhc1V0aWwKUGFuZGFzU2NoZW1hCnEAKYFxAX1xAihYDAAAAGNvbHVtbl9uYW1lc3EDXXEEKFgGAAAAcHJlY2lwcQVYBAAAAHRlbXBxBlgEAAAAaG91cnEHWAUAAABtb250aHEIWAkAAABkYXlvZndlZWtxCVgLAAAAZGVtYW5kX2xhZzFxClgLAAAAZGVtYW5kX2xhZzJxC1gLAAAAZGVtYW5kX2xhZzNxDFgLAAAAZGVtYW5kX2xhZzRxDVgLAAAAZGVtYW5kX2xhZzVxDlgLAAAAZGVtYW5kX2xhZzZxD1gMAAAAZGVtYW5kX2xhZzEycRBYDAAAAGRlbWFuZF9sYWcyNHERWBAAAABkZW1hbmRfcm1lYW5fZGF5cRJYDwAAAGRlbWFuZF9ybWVhbl93a3ETWA4AAAB0ZW1wX3JtZWFuX2RheXEUWA0AAAB0ZW1wX3JtZWFuX3drcRVlWAwAAABjb2x1bW5fdHlwZXNxFl1xFyhjbnVtcHkKZHR5cGUKcRhYAgAAAGY4cRlLAEsBh3EaUnEbKEsDWAEAAAA8cRxOTk5K/////0r/////SwB0cR1iaBtoGFgCAAAAaThxHksASwGHcR9ScSAoSwNoHE5OTkr/////Sv////9LAHRxIWJoIGggaBtoG2gbaBtoG2gbaBtoG2gbaBtoG2gbZVgFAAAAc2hhcGVxIksBSxGGcSN1Yi4=',
   'swagger': {'example': [{'dayofweek': 4,
      'demand_lag1': 6912.7,
      'demand_lag12': 8161.383,
      'demand_lag2': 7332.625,
      'demand_lag24': 6247.433,
      'demand_lag3': 7576.558,
      'demand

### Deploy the web service
Open the Azure Machine Learning command-line interface under the File menu. Run the following commands to deploy the web service to your local machine.

> az ml service create realtime -f score.py -m ../ -s service_schema.json -n demandforecast -r python